# Generic File Processor with Autoloader
# Handles both ZIP archives (extract contents) and regular documents (direct catalog)

This notebook:

- Uses **Auto Loader** to ingest files from a Unity Catalog Volume.
- Routes files based on extension:
  - **Archives** (.zip, .tar.gz, etc.) → Extract contents recursively
  - **Regular files** (.pdf, .xlsx, etc.) → Catalog directly
- Writes a **Delta manifest table** listing all processed files with metadata.
- Supports configurable source/target paths via workflow parameters.


In [0]:
import json
from clinical_data_standards_framework.utils import (
    create_zip_autoloader_stream,
    create_zip_batch_processor,
    save_with_audit,
    get_manifest_schema,
    get_document_status_schema
)


{'source_root_dbfs': 'dbfs:/Volumes/aira/bronze_md/input_data/historical_data/zip',
 'target_root_dbfs': 'dbfs:/Volumes/aira/bronze_md/input_data/historical_data/extracted',
 'checkpoint_path': '/Volumes/aira/bronze_md/input_data/jobs/_checkpoints/zip_processor',
 'manifest_table': 'aira.bronze_md.trial_data_history'}

In [ ]:
# Get configuration from setup task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
services_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="services"))
pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")


In [ ]:
# Extract needed values from globals
catalog = globals_dict['catalog']
schema = globals_dict['bronze_schema']

# Get file processor configuration from pipeline_config
# New flatter structure - no more nested archives/clinical_trial_data/extraction
manifest_table = pipeline_config['manifest_table']
manifest_table_description = pipeline_config.get('manifest_table_description', '')
status_table = pipeline_config['status_table']
status_table_description = pipeline_config.get('status_table_description', '')
checkpoint_subdir = pipeline_config['checkpoint_subdir']
supported_extensions = pipeline_config['supported_extensions']

# Get document categorization config
document_categories = pipeline_config.get('document_categories', {})

# Get file type definitions (for routing logic)
file_types = pipeline_config.get('file_types', {})
archive_extensions = file_types.get('archives', {}).get('extensions', ['.zip'])
document_extensions = file_types.get('documents', {}).get('extensions', ['.pdf', '.xlsx'])

print(f"\n📋 File Processor Configuration:")
print(f"  Catalog: {catalog}")
print(f"  Schema: {schema}")
print(f"  Manifest Table: {manifest_table}")
print(f"  Status Table: {status_table}")
print(f"  Archive extensions: {archive_extensions}")
print(f"  Document extensions: {document_extensions}")
print(f"  Supported extensions (from extracted): {supported_extensions}")
print(f"  Document categories: {len(document_categories)} configured")
print(f"\n✅ Config loaded - paths will be computed from setup task values")


In [ ]:
# Read source/target paths from setup task (computed from workflow params or config defaults)
source_volume = dbutils.jobs.taskValues.get(taskKey="setup", key="source_volume")
source_subdir = dbutils.jobs.taskValues.get(taskKey="setup", key="source_subdir")
target_volume = dbutils.jobs.taskValues.get(taskKey="setup", key="target_volume")
target_subdir = dbutils.jobs.taskValues.get(taskKey="setup", key="target_subdir")

# Construct full paths
source_root = f"/Volumes/{catalog}/{schema}/{source_volume}/{source_subdir}"
target_root = f"/Volumes/{catalog}/{schema}/{target_volume}/{target_subdir}"
manifest_table_full_name = f"{catalog}.{schema}.{manifest_table}"
status_table_full_name = f"{catalog}.{schema}.{status_table}"

# Create source-specific checkpoint path to avoid conflicts when processing different sources
# Replace slashes with underscores to create a safe directory name
source_identifier = source_subdir.replace('/', '_')
checkpoint_path = f"/Volumes/{catalog}/{schema}/{source_volume}/{checkpoint_subdir}_{source_identifier}"

print(f"\n📁 File Processor Paths:")
print(f"  Source root: {source_root}")
print(f"  Target root: {target_root}")
print(f"  Manifest table: {manifest_table_full_name}")
print(f"  Status table: {status_table_full_name}")
print(f"  Checkpoint: {checkpoint_path}")
print(f"    (unique for source: {source_subdir})")


In [0]:
# Create Auto Loader streaming DataFrame for all files
# Reads both archives (.zip, .tar.gz, etc.) and regular files (.pdf, .xlsx, etc.)
file_stream_df = create_zip_autoloader_stream(spark, source_root)

# Display schema
file_stream_df.printSchema()


root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)



In [0]:
# Create generic file batch processor with routing logic
# Routes files based on extension: ZIP → extract, regular → catalog

# Create ZIP-specific processor
zip_batch_processor = create_zip_batch_processor(
    spark=spark,
    target_root_path=target_root,
    manifest_table_full_name=manifest_table_full_name,
    status_table_full_name=status_table_full_name,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id,
    supported_extensions=supported_extensions,
    categories_config=document_categories,
    manifest_table_description=manifest_table_description,
    status_table_description=status_table_description
)

# Wrapper function for file routing
def process_file_batch(batch_df, batch_id):
    """Route files based on extension: archives vs regular documents"""
    import os
    
    print(f"\n{'='*80}")
    print(f"Processing batch {batch_id}")
    print(f"{'='*80}")
    
    # Collect file paths and classify
    files = batch_df.select("path").collect()
    archive_files = []
    regular_files = []
    
    for file_row in files:
        file_path = file_row.path
        ext = os.path.splitext(file_path.lower())[1]
        
        if ext in archive_extensions:
            archive_files.append(file_path)
        elif ext in document_extensions:
            regular_files.append(file_path)
        else:
            print(f"  ⚠ Skipping unsupported file type: {file_path} (ext: {ext})")
    
    print(f"Files in batch: {len(files)}")
    print(f"  Archives (.zip, etc.): {len(archive_files)}")
    print(f"  Documents (.pdf, .xlsx, etc.): {len(regular_files)}")
    
    # Process archives (if any)
    if archive_files:
        print(f"\n→ Processing {len(archive_files)} archive file(s)...")
        archive_df = batch_df.filter(batch_df.path.isin(archive_files))
        zip_batch_processor(archive_df, batch_id)
    
    # Process regular files (if any)
    if regular_files:
        print(f"\n→ Processing {len(regular_files)} regular file(s)...")
        regular_df = batch_df.filter(batch_df.path.isin(regular_files))
        process_regular_files(regular_df, batch_id)
    
    print(f"✓ Batch {batch_id} complete")
    print(f"{'='*80}")

print("Generic file batch processor created successfully")
print(f"Archive extensions: {archive_extensions}")
print(f"Document extensions: {document_extensions}")
print(f"Supported file extensions (from extracted contents): {supported_extensions}")
print(f"Status tracking enabled - writing to: {status_table_full_name}")
if document_categories:
    print(f"Document categorization enabled with {len(document_categories)} top-level categories")
else:
    print("Document categorization not configured")


In [ ]:
# Cell 7: Regular File Processing Function
# Handles non-archive files (PDF, Excel, etc.) using save_document_with_metadata with versioning
# Uses save_with_audit for automatic audit column management

def process_regular_files(regular_df, batch_id):
    """
    Process regular files (non-archives) using the same pattern as ZIP extraction.
    
    Supports versioning: if the same file is uploaded multiple times:
    - Old version is marked as is_current=false
    - New version gets incremented version number and supersedes_document_id link
    
    NOTE: Audit columns are added automatically by save_with_audit() - 
          DO NOT add them manually to records!
    """
    import os
    from datetime import datetime
    from clinical_data_standards_framework.utils import (
        save_document_with_metadata,
        save_with_audit,
        get_manifest_schema,
        get_document_status_schema
    )
    
    # Collect file info WITH binary content (already loaded by Autoloader)
    files = regular_df.select("path", "modificationTime", "length", "content").collect()
    
    if not files:
        print("  No regular files to process")
        return
    
    # Process each file using the unified function
    manifest_records = []
    status_records = []
    old_version_ids = []  # Track old versions to mark as not current
    
    for file_row in files:
        source_path = file_row.path
        content_bytes = file_row.content  # ✅ Binary content already loaded by Autoloader!
        file_size = file_row.length
        file_name = os.path.basename(source_path)
        
        # Compute target path (maintain same folder structure)
        source_path_clean = source_path.replace('dbfs:', '')
        relative_path = source_path_clean.replace(source_root, '').lstrip('/')
        target_path = f"{target_root}/{relative_path}"
        
        try:
            # Use the unified save_document_with_metadata function
            # This handles file write, categorization, base64 encoding, versioning, and metadata creation
            # NOTE: Returns feature columns only - audit columns added by save_with_audit
            manifest_record = save_document_with_metadata(
                content_bytes=content_bytes,
                source_path=source_path,
                target_path=target_path,
                file_size=file_size,
                archive_root=target_root,
                categories_config=document_categories,
                created_by_principal=created_by_principal,
                databricks_job_id=databricks_job_id,
                databricks_job_name=databricks_job_name,
                databricks_run_id=databricks_run_id,
                is_from_nested_zip=False,
                nested_level=0,
                spark=spark,  # Pass spark for version checking
                manifest_table_full_name=manifest_table_full_name
            )
            
            manifest_records.append(manifest_record)
            
            # Track if this is an update (has supersedes_document_id)
            if manifest_record.get('supersedes_document_id'):
                old_version_ids.append(manifest_record['supersedes_document_id'])
                print(f"  ✓ Updated: {file_name} (v{manifest_record['version']})")
            else:
                print(f"  ✓ Processed: {file_name} (v1)")
            
            # Create corresponding status record (FEATURE COLUMNS ONLY)
            # Audit columns are added automatically by save_with_audit()
            status_records.append({
                'document_id': manifest_record['document_id'],
                'status': 'READY_FOR_PROCESSING',
                'status_timestamp': datetime.now(),
                'error_message': None,
                'processing_duration_seconds': None,
                'retry_count': 0,
                'processing_metadata': None
            })
            
        except Exception as e:
            print(f"  ✗ ERROR processing file {file_name}: {str(e)}")
            # Still create a status record to track the error (FEATURE COLUMNS ONLY)
            import uuid
            doc_id = str(uuid.uuid4())
            status_records.append({
                'document_id': doc_id,
                'status': 'FILE_PROCESSING_ERROR',
                'status_timestamp': datetime.now(),
                'error_message': str(e),
                'processing_duration_seconds': None,
                'retry_count': 0,
                'processing_metadata': None
            })
    
    # Write to Delta tables using save_with_audit (adds audit columns automatically)
    if manifest_records or status_records:
        if manifest_records:
            manifest_df = spark.createDataFrame(manifest_records, schema=get_manifest_schema())
            # Use save_with_audit - it adds audit columns automatically
            save_with_audit(
                df=manifest_df,
                table_name=manifest_table_full_name,
                created_by_principal=created_by_principal,
                databricks_job_id=databricks_job_id,
                databricks_job_name=databricks_job_name,
                databricks_run_id=databricks_run_id,
                mode='append'
            )
            print(f"  ✓ Saved {len(manifest_records)} document(s) to manifest table")
        
        if status_records:
            status_df = spark.createDataFrame(status_records, schema=get_document_status_schema())
            # Use save_with_audit - it adds audit columns automatically
            save_with_audit(
                df=status_df,
                table_name=status_table_full_name,
                created_by_principal=created_by_principal,
                databricks_job_id=databricks_job_id,
                databricks_job_name=databricks_job_name,
                databricks_run_id=databricks_run_id,
                mode='append'
            )
            print(f"  ✓ Created {len(status_records)} status record(s)")
        
        # Mark old versions as not current (if any updates occurred)
        if old_version_ids:
            ids_list = ', '.join([f"'{id}'" for id in old_version_ids])
            spark.sql(f"""
                UPDATE {manifest_table_full_name}
                SET is_current = false,
                    last_updated_by_principal = '{created_by_principal}',
                    last_updated_ts = current_timestamp()
                WHERE document_id IN ({ids_list})
            """)
            print(f"  ✓ Marked {len(old_version_ids)} old version(s) as not current")

print("✅ Regular file processor function created (uses save_with_audit for automatic audit columns)")


In [0]:
# Start the Auto Loader streaming query
# For one-time historical processing, use availableNow=True
# For continuous ingestion, change trigger to processingTime or remove trigger entirely

query = (
    file_stream_df.writeStream
                  .foreachBatch(process_file_batch)  # ← Generic processor (routes to ZIP or regular)
                  .option('checkpointLocation', checkpoint_path)
                  .trigger(availableNow=True)  # change for continuous streaming if desired
                  .start()
)

query.awaitTermination()


In [0]:
# Cell 8: Conditional OPTIMIZE - trial_data_history table
# Only run OPTIMIZE when conditions are met to save compute resources

from datetime import datetime

# Get table stats
try:
    table_stats = spark.sql(f"DESCRIBE DETAIL {manifest_table_full_name}").first()
    num_files = table_stats.numFiles
    total_size_gb = table_stats.sizeInBytes / (1024**3)
    avg_file_size_mb = (total_size_gb * 1024) / num_files if num_files > 0 else 0
    
    print(f"\n{'='*80}")
    print(f"TABLE STATS: {manifest_table_full_name}")
    print(f"{'='*80}")
    print(f"Total files: {num_files}")
    print(f"Total size: {total_size_gb:.2f} GB")
    print(f"Avg file size: {avg_file_size_mb:.2f} MB")
    
    # Determine if OPTIMIZE is needed
    should_optimize = False
    reason = ""
    
    # We don't know exact doc count processed, so use file-based heuristics
    if num_files > 200 and avg_file_size_mb < 100:  # Condition 1: Small file problem
        should_optimize = True
        reason = f"Small file problem ({num_files} files, avg {avg_file_size_mb:.2f}MB)"
    elif num_files > 500:  # Condition 2: Too many files
        should_optimize = True
        reason = f"Too many files ({num_files})"
    elif total_size_gb > 1 and num_files > 100 and avg_file_size_mb < 200:  # Condition 3: Growing dataset with small files
        should_optimize = True
        reason = f"Dataset growing with sub-optimal file sizes ({total_size_gb:.2f}GB, {num_files} files)"
    
    if should_optimize:
        print(f"\n🔧 OPTIMIZE TRIGGERED: {reason}")
        print(f"{'='*80}")
        
        start_time = datetime.now()
        
        # Run OPTIMIZE with Z-ORDER
        spark.sql(f"""
            OPTIMIZE {manifest_table_full_name}
            ZORDER BY (current_status, document_tags)
        """)
        
        # Get stats after optimization
        table_stats_after = spark.sql(f"DESCRIBE DETAIL {manifest_table_full_name}").first()
        num_files_after = table_stats_after.numFiles
        
        duration = (datetime.now() - start_time).total_seconds()
        
        print(f"✓ OPTIMIZE completed in {duration:.1f} seconds")
        print(f"  Files: {num_files} → {num_files_after} ({num_files - num_files_after} merged)")
        print(f"{'='*80}")
    else:
        print(f"\n⏭️  OPTIMIZE SKIPPED: Conditions not met")
        print(f"  - Files: {num_files} (threshold: >200 with small files OR >500 total)")
        print(f"  - Avg file size: {avg_file_size_mb:.2f}MB (threshold: <100-200MB)")
        print(f"  - Total size: {total_size_gb:.2f}GB")
        print(f"{'='*80}")
        
except Exception as e:
    print(f"\n⚠️  Could not check OPTIMIZE conditions: {str(e)}")
    print("Continuing without optimization...")